# Lab 4.2 Using Python for Data Collection


## 2.1 Python programming for data collection from power meter


The entire schematic of the data communication using Modbus protocol in case of multiple masters 
(computers) and a single slave (power meter) is shown in Figure 10. In the lab, your computers (laptop and Raspberry Pi) are on the router network. The Modbus RTU to Modbus TCP converter (gateway) is also on the same network with a static IP address. The ethernet cable (RJ45) is used for connection between the router and the gateway. The data communication sequence to read data from power meter is as follows. 

1.	The computer (Raspberry Pi or laptop) requests read (data) holding registers using Modbus TCP protocol via the router to the gateway. 
2.	The gateway converts Modbus TCP packet frame to Modbus RTU data frame and sends it to the slave. 
3.	The slave responds to the request and sends Modbus RTU data frame to the gateway. 
4.	The gateway converts the received Modbus RTU data frame to Modbus TCP packet frame. 
5.	The gateway sends the converted packet to a specific IP host which is the computer requested data. 
6.	The computer receives Modbus TCP packet frame and interprets the packet to the data. 

To do this, we will use Python and ‘pymodbus’ module. If you are interested in ‘pymodbus’ module in detail, please visit the website (https://pymodbus.readthedocs.io/en/latest/, available on Feb. 3, 2022).


![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure10.png?raw=true)

*Figure 10 Schematic of communication for power meter data collection*

What you need to do first to use TCP/IP communication and application is to check network connectivity. By using ping test from Raspberry Pi, check the network connection between Raspberry Pi and the gateway. TA will let you know the IP address of the gateway at Lab4. 




### Task 2.1

Capture the result of checking network connection using ping test from Raspberry Pi to the gateway for Modbus as Figure 11 and attach it to the report. 
*  Please note that the IP address of the gateway will be different from the example.


![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure11.png?raw=true)

*Figure 11 Terminal window after ping test from Raspberry Pi to the gateway*

To read data from the power meter, we need to request to read registers via Modbus protocol. The selected primary data address of basic measurement of the power meter is shown in Table 4. All Modbus map is on the AccuEnergy website (https://www.accuenergy.com/support/modbus-map/?modbus-map=acuvim-l-modbusmap, available on Feb. 3, 2022). Or refer to Table 5-16 on page 109 of the manual (Acuvim L User’s Manual.pdf on Brightspace). Please note that the minimum refresh rate of holding registers of the power meter is about 250 msec which means you can collect 4 times per second. Therefore, your request interval should be larger than 250 msec. 

*Table 4 Selected primary data address of basic measurement of the power meter*

<table width="671">
<tbody>
<tr>
<td width="106">
<p>Address (Hex)</p>
</td>
<td width="89">
<p>Address (Decimal)</p>
</td>
<td width="58">
<p>Symbol</p>
</td>
<td width="155">
<p>Parameter</p>
</td>
<td width="120">
<p>Range</p>
</td>
<td width="50">
<p>Unit</p>
</td>
<td width="68">
<p>Data Type</p>
</td>
<td width="110">
<p>Access Property</p>
</td>
</tr>
<tr>
<td width="106">
<p>0600H-0601H</p>
</td>
<td width="89">
<p>1536-1537</p>
</td>
<td width="58">
<p>F</p>
</td>
<td width="148">
<p>Frequency</p>
</td>
<td width="110">
<p>4500 ~ 6500</p>
</td>
<td width="40">
<p>Hz</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
<tr>
<td width="106">
<p>0602H-0603H</p>
</td>
<td width="89">
<p>1538-1538</p>
</td>
<td width="58">
<p>U1</p>
</td>
<td width="148">
<p>Phase 1 Voltage</p>
</td>
<td width="110">
<p>0 ~ 65535</p>
</td>
<td width="40">
<p>V</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
<tr>
<td width="106">
<p>060EH-060FH</p>
</td>
<td width="89">
<p>1550-1551</p>
</td>
<td width="58">
<p>I1</p>
</td>
<td width="148">
<p>Total Phase A Current</p>
</td>
<td width="110">
<p>0 ~ 65535</p>
</td>
<td width="40">
<p>A</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
<tr>
<td width="106">
<p>061CH-061DH</p>
</td>
<td width="89">
<p>1564-1565</p>
</td>
<td width="58">
<p>Psum</p>
</td>
<td width="148">
<p>Total System power</p>
</td>
<td width="110">
<p>-32768 ~ 32768</p>
</td>
<td width="40">
<p>W</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
<tr>
<td width="106">
<p>0624H-0623H</p>
</td>
<td width="89">
<p>1572-1573</p>
</td>
<td width="58">
<p>Qsum</p>
</td>
<td width="148">
<p>Total Reactive power</p>
</td>
<td width="110">
<p>-32768 ~ 32768</p>
</td>
<td width="40">
<p>var</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
<tr>
<td width="106">
<p>0626H-0627H</p>
</td>
<td width="89">
<p>1574-1575</p>
</td>
<td width="58">
<p>Ssum</p>
</td>
<td width="148">
<p>Total Apparent Power</p>
</td>
<td width="110">
<p>0 ~ 65535</p>
</td>
<td width="40">
<p>VA</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
<tr>
<td width="106">
<p>0628EH-062FH</p>
</td>
<td width="89">
<p>1582-1583</p>
</td>
<td width="58">
<p>PFsum</p>
</td>
<td width="148">
<p>Total Power Factor</p>
</td>
<td width="110">
<p>-1000 ~ 1000</p>
</td>
<td width="40">
<p>-</p>
</td>
<td width="48">
<p>float</p>
</td>
<td width="72">
<p>R</p>
</td>
</tr>
</tbody>
</table>

---

**Python - Python 3 (lab4_sample1.py)**

```
from pymodbus.client.sync import ModbusTcpClient
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder
import datetime

# define the gateway IP (host) and port
# normally Modbus TCP uses port number 502
host = "192.168.1.x" # TA will let you know the IP address of the gateway.
port = 502

# define client object of the host and port
client = ModbusTcpClient(host, port)
client.connect()

now = datetime.datetime.now()

# read the holding registers for frequency
# Refer to the Modbus address map
# the arguments are register starting register address (decimal), length, and unit=1.
freq_read = client.read_holding_registers(1536, 2, unit=1)

# get received registers
freq_reg = freq_read.registers

# define decode the received register values
freq_decoder = BinaryPayloadDecoder.fromRegisters(freq_reg, byteorder=Endian.Big, wordorder=Endian.Big)

# get the frequency value by decoding it to 32bit float data.
freq_value = freq_decoder.decode_32bit_float()

# print out value
print("{}: Frequency is {} Hz".format(now, freq_value))

# gently close the client object.
client.close()

```

---

The sample code 1 (lab4_sample1.py on Brightspace) is given as above. This code is to print out a timestamp and the measured value of frequency from the power meter as Figure 12. Please refer to the comments on the script for the descriptions. Please note that the read values of registers by Modbus protocol are encoded and it may be differed by the device. In this case, the register values are decoded by Big Endian Byte order. For the details of understanding Byte order encoding and decoding, please visit the website (https://www.digitaldetective.net/understanding-big-and-little-endian-byte-order/, available on Feb. 3, 2022). 

### Task 2.2

Model the code block above (lab4_sample1.py) by creating a flow diagram that illustrates the logic, sequence, and processes of the code

---

Place URL here

---

### Task 2.3

Run the sample code 1 and then capture and attach the result as Figure 12.
* Capture either one of terminal window as Figure 12 (left) or Thonny IDE Shell as Figure 12 (right). 
* Try this on your laptop and see if it works as well. 

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure12.png?raw=true)

*Figure 12 Results of running sample code 1: terminal window (left) and Thonny IDE shell (right)*

### Task 2.4

1.	Modify sample code 1 to collect data for at least 1 minute and save data as a CSV file and upload the CSV file on Brightspace. 
* Hint: Make function (method) to convert received registers to a float value. 
* Set the sampling period 1 second. (Hint: use ‘while’ loop and ‘time’ package) 
* Data (header) should include time, frequency, voltage, current, power factor, and true power as Table 5. 
* Refer to the sample codes in previous lab manuals. 
2.	Plot each column data according to time as Figure 13 and attach all plots to the report. 

*Table 5 Example data of output CSV file for TASK 4*

<table width="671">
<tbody>
<tr>
<td width="90">
<p>Time [sec]</p>
</td>
<td width="114">
<p>Frequency [Hz]</p>
</td>
<td width="102">
<p>Voltage 1 [V]</p>
</td>
<td width="108">
<p>Current 1 [A]</p>
</td>
<td width="126">
<p>Power factor [-]</p>
</td>
<td width="132">
<p>True power [W]</p>
</td>
</tr>
<tr>
<td width="90">
<p>&nbsp;</p>
</td>
<td width="114">
<p>&nbsp;</p>
</td>
<td width="102">
<p>&nbsp;</p>
</td>
<td width="108">
<p>&hellip;</p>
</td>
<td width="126">
<p>&nbsp;</p>
</td>
<td width="132">
<p>&nbsp;</p>
</td>
</tr>
<tr>
<td width="90">
<p>30.17739</p>
</td>
<td width="114">
<p>60.0008443</p>
</td>
<td width="102">
<p>119.73915</p>
</td>
<td width="108">
<p>2.481679</p>
</td>
<td width="126">
<p>0.744922</p>
</td>
<td width="132">
<p>221.9443</p>
</td>
</tr>
<tr>
<td width="90">
<p>31.204921</p>
</td>
<td width="114">
<p>60.0007994</p>
</td>
<td width="102">
<p>120.011049</p>
</td>
<td width="108">
<p>2.452804</p>
</td>
<td width="126">
<p>0.737972</p>
</td>
<td width="132">
<p>221.4492</p>
</td>
</tr>
<tr>
<td width="90">
<p>&nbsp;</p>
</td>
<td width="114">
<p>&nbsp;</p>
</td>
<td width="102">
<p>&nbsp;</p>
</td>
<td width="108">
<p>&hellip;</p>
</td>
<td width="126">
<p>&nbsp;</p>
</td>
<td width="132">
<p>&nbsp;</p>
</td>
</tr>
</tbody>
</table>
<p>&nbsp;</p>

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure13.png?raw=true)

*Figure 13 Voltage vs. time plot example for TASK 4*

Please continue to [Lab 4.3 here](L4_Colab3.ipynb).
